<a href="https://colab.research.google.com/github/flavianacif/DSWP/blob/master/RFB_RN_59_GRIDSEARCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Use scikit-learn to grid search the number of neurons
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm


In [52]:
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [48]:
import sklearn

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import tensorflow_addons as tfa

from tensorflow import keras

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt # importing ploting libraries
import seaborn as sns # importing seaborn for statistical plots
from collections import Counter

from sklearn.metrics import make_scorer,accuracy_score, f1_score, confusion_matrix, classification_report, recall_score # calculate accuracy measures and confusion matrix

from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


[**Python**] - Verificar a versão do Tensorflow
> Assegurar que está a utilizar a versão 2.x.

In [3]:
tf.__version__

'2.3.0'

[**Python**] - Definir o número de casas decimais

In [4]:
np.set_printoptions(precision= 3)

### 1. Carregar os dados

[**Python**] - Carregar os dados:

In [5]:
# Leitura do dataframe:
df = pd.read_csv('/total_9.csv')

In [6]:
df_total = df.copy()

In [7]:
df_total.set_index('id', inplace=True)

Retirando algumas colunas para testar

In [9]:
df_total.drop(columns = ['cnae2','md1', 'md2', 'md3', 'md4', 'md5', 'md6', 'md7', 'md8',
       'md9', 'md10', 'md11', 'md12', 'mc1', 'mc2', 'mc3', 'mc4'], inplace = True)

GET DUMMIES

In [10]:
df_total = pd.get_dummies(df_total)

In [11]:
X_treino = df_total[df_total['target']!= 2]

In [13]:
df_X = X_treino.copy()

In [14]:
df_y = df_X['target']
df_X = df_X.drop(columns= ['target'])

In [15]:
X_test = df_total[df_total['target']== 2].drop(columns= ['target'])

Tratando o Balanceamento da Amostra

In [16]:
qtd = Counter(X_treino['target'])
qtd

Counter({0: 8513, 1: 2520})

In [17]:
np.round(100*qtd[1]/qtd[0], 4)

29.6018

In [18]:
from imblearn.under_sampling import RandomUnderSampler

In [19]:
over = SMOTE(sampling_strategy = 0.5) # Reamostrar a classe minoritária para ter 10% da classe majoritária
under = RandomUnderSampler(sampling_strategy = 0.85) # Seleciona a classe majoritária para ter 50%

Usando um Pileline

In [20]:
from imblearn.pipeline import Pipeline

In [22]:
steps = [('over', over), ('under', under)]
pipeline = Pipeline(steps = steps)

X_cc e y_cc são nossos dataframes originais.

In [23]:
# Aplica o pipeline
X, y = pipeline.fit_resample(df_X, df_y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Antes:

In [24]:
Counter(df_y)

Counter({0: 8513, 1: 2520})

Depois:

In [25]:
qtd_d = Counter(y)
qtd_d

Counter({0: 5007, 1: 4256})

In [90]:

# Function to create model, required for KerasClassifier
def create_model(neurons, epochs,batch_size,n_hl):
  # create model
  model = Sequential()
  model.add(Dense(neurons, input_dim=88, kernel_initializer='glorot_normal', activation='relu'))
  for i in np.arange(0,n_hl):
    model.add(Dropout(0.1))
    neurons = round(neurons/2)
    model.add(Dense(neurons, kernel_initializer='glorot_normal', activation='relu'))
  # Compile model
  model.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='Adamax', metrics= ['accuracy'])
  return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# split into input (X) and output (Y) variables
X_2 = np.array(X)
y_2 = y


In [94]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=1)
# define the grid search parameters
batch_size = [20]
epochs = [70]
neurons = [600]
n_hl = [7]
param_grid = dict(batch_size=batch_size, epochs=epochs, neurons=neurons, n_hl = n_hl)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=10,scoring='f1_macro')
grid_result = grid.fit(X_2, y_2)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/70
464/464 [==============================] - 2s 5ms/step - loss: 0.6368 - accuracy: 0.6201
Epoch 2/70
464/464 [==============================] - 2s 5ms/step - loss: 0.6070 - accuracy: 0.6620
Epoch 3/70
464/464 [==============================] - 2s 5ms/step - loss: 0.5929 - accuracy: 0.6773
Epoch 4/70
464/464 [==============================] - 2s 5ms/step - loss: 0.5829 - accuracy: 0.6906
Epoch 5/70
464/464 [==============================] - 2s 5ms/step - loss: 0.5712 - accuracy: 0.6990
Epoch 6/70
464/464 [==============================] - 2s 5ms/step - loss: 0.5590 - accuracy: 0.7069
Epoch 7/70
464/464 [==============================] - 2s 5ms/step - loss: 0.5456 - accuracy: 0.7262
Epoch 8/70
464/464 [==============================] - 2s 5ms/step - loss: 0.5297 - accuracy: 0.7329
Epoch 9/70
464/464 [==============================] - 2s 5ms/step - loss: 0.5171 - accuracy: 0.7468
Epoch 10/70
464/464 [==============================] - 2s 5ms/step - loss: 0.4998 - accuracy: 0.7603

In [74]:
keras.activations.

TypeError: ignored